In [1]:
import numpy as np

In [2]:
import xgboost as xgb

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [4]:
features = pd.read_csv('features_09.csv')
kaggle = pd.read_csv('trocafone_kaggle_test.csv')
labels = pd.read_csv('labels_training_set.csv')

In [5]:
training = labels.set_index('person').join(features.set_index('person'))
train_labels = training['label']
train_data = training.drop('label',axis=1)

In [6]:
predict_data = kaggle.set_index('person').join(features.set_index('person'))

In [7]:
encoder = category_encoders.TargetEncoder(smoothing=True)
train_data_enc = encoder.fit_transform(np.array(train_data),np.array(train_labels))
predict_data_enc = encoder.transform(np.array(predict_data))

In [8]:
xg_classifier = XGBClassifier()

NameError: name 'XGBClassifier' is not defined

In [9]:
xtrain,xtest,ytrain,ytest = train_test_split(train_data_enc,np.array(train_labels))

In [10]:
train = xgb.DMatrix(data=xtrain,label=ytrain)
test = xgb.DMatrix(data=xtest,label=ytest)
data = xgb.DMatrix(data=train_data_enc,label=np.array(train_labels))
pred = xgb.DMatrix(data=predict_data_enc)
params = {'colsample_bytree': 0.9,
 'gamma': 2,
 'learning_rate': 0.016238752458245277,
 'max_depth': 13,
 'min_child_weight': 5,
 'n_estimators': 100,
 'subsample': 0.7847190225361189,
 'eval_metric':'auc'}

In [11]:
bst = xgb.train(params,train,200,[(train,'train'),(test,'eval')])

[09:22:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 336 pruned nodes, max_depth=11
[0]	train-auc:0.829207	eval-auc:0.767766
[09:22:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 314 pruned nodes, max_depth=10
[1]	train-auc:0.850653	eval-auc:0.79841
[09:22:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 372 pruned nodes, max_depth=11
[2]	train-auc:0.873757	eval-auc:0.821145
[09:22:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 350 pruned nodes, max_depth=11
[3]	train-auc:0.873508	eval-auc:0.819403
[09:22:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 280 pruned nodes, max_depth=13
[4]	train-auc:0.881388	eval-auc:0.819286
[09:22:42] C:\Users\Admini

In [12]:
bst = xgb.train(params,data,200)

[09:23:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 402 pruned nodes, max_depth=13
[09:23:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 436 pruned nodes, max_depth=11
[09:23:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 380 pruned nodes, max_depth=11
[09:23:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 432 pruned nodes, max_depth=11
[09:23:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 484 pruned nodes, max_depth=8
[09:23:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 444 pruned nodes, max_depth=9
[09:23:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:7

In [13]:
xg_prediction = bst.predict(pred)

In [14]:
submit = kaggle.set_index('person')

In [15]:
submit['label'] = xg_prediction

In [84]:
submit.to_csv('submit.csv')

In [97]:
bst.save_model('86605_xgboost_combinar_con_86602_catboost')

## Combinando con catboost (0.86605)

In [16]:
import catboost as cb

In [17]:
cb_classifier = cb.CatBoostClassifier(iterations=400,
                                     depth=8,
                                     l2_leaf_reg=100,
                                     random_strength=0.1,
                                     bagging_temperature=0,
                                     border_count=254,
                                     learning_rate=0.03)

In [19]:
cb_classifier.load_model('preentrenados/86602catboost')

In [20]:
cb_prediction = cb_classifier.predict_proba(predict_data)[:,1]

In [21]:
prediction = 0.9*cb_prediction+0.1*xg_prediction

In [22]:
submit = kaggle.set_index('person')

In [23]:
submit['label'] = prediction

In [24]:
submit.to_csv('submit.csv')